# Programming for Data Science
## NOVA IMS

#### Group Name/Number: <br>
1. María Fernanda Restrepo Suescún / <span style="color:blue"> Num: 20190679 <br>
2. Rennan Valadares Ornelas Araújo / <span style="color:blue"> Num: 20190146 <br>
3. Henrique Confraria Varatojo Caldas Januário / <span style="color:blue"> Num: 20190438 <br>
4. Chanmolyta Tham  / <span style="color:blue"> Num: 20190927 <br>


### Group Project Summary
This project report aims to present insight into a potential client and he is willing to know where he should invest next, particularly in the tourism sector. The key recommendations to business's need based on the cost and risk analysis, and other factors including socio-economic, political, and technology potential, etc. We will get into this in the plots, where countries with high-growth, high-risk, high-arrivals, high competition and On-the-Growth with medium/low risk. 

The dataset we used is International Tourism by Number of Arrivals by country, the world, data sourced from the World Tourism Organisation (WTO) and the World Bank's databank.


Our group starts checking the provided dataset, tried to understand the regionals and countries' perspectives and its currents status, countries made up to the top of the list as well as countries had the remarkable growth in the past 10 years. Also, businesses' ideas in the Tourism sector, that link within/or the rest of the regions, only within countries kind of business.

***We used GitHub from the beginning to version control. You can check our project on the following link: https://github.com/rennanvoa2/Programming_DS_Project***

The following report presents a step-by-step process from data preparation to data visualization and presenting. 


### 1. Data Cleaning 

#### 1.1 Importing libraries

We used Scikit Learn for Machine Learning models for filling the NaNs with a regression, pandas, and NumPy for managing data, MatPlotLib and Seaborn for plotting.

In [1]:
#remove warnings
import warnings
warnings.filterwarnings("ignore")


#Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from copy import deepcopy
from sklearn.svm import LinearSVR
import seaborn as sb

#### 1.2 Functions for Data Preparation

##### 1.2.1 regressor_test
We started filling our NaNs with the Average of each column. Then we figured out that we could create a regressor for having better results.<br>
The main goal for the regressor_test function is to realize which of the 3 Machine Learning models is the best one for filling our NaN data. To accomplish this the regressor_test function create's 3 different regressor models ***for each column*** following these steps:<br><br>
<span style="color:blue">
1 - Split the data in Train and Test (80% and 20% respectively)<br>
2 - Create 3 different regressors: KNeighborsRegressor, LinearRegression, and LinearSVR.<br>
3 - Train each regressor with Train data.<br>
4 - Fill other columns (instead of the one in the loop) with the row average. We decided to do it because we cant have NaNs on the other columns.<br>
5 - Make the Predictions on the test set.<br>
6 - Check the Mean Squared Error for each model and append it to its list.<br>
7 - Append the best model for each column on the MSE list.<br>
8 - Count number of times of each model name appears on the list and return the highest.<br>


In [2]:
def regressor_test(complete,incomplete,years):
    '''This function test 3 diferent Machine Learning models (KNeighborsRegressor, LinearRegression, LinearSVR)
    returning the best one.
    It will return a string with the name of the best model.
    Complete: Pandas Dataset with complete data.
    Incomplete: Pandas Dataset with NaNs.
    years: List with the name of the columns'''
    kn_errors = []
    linear_errors = []
    svr_errors = []    
    
    for i in years:
        
        #Step 1
        X_train, X_test, y_train, y_test = train_test_split(complete.loc[:,complete.columns != i].values,
                                                            complete.loc[:,i].values, test_size = 0.2, random_state = 0)
        
        #Step 2
        regressor1 = KNeighborsRegressor(2, 
                                       weights ='distance', 
                                       metric = 'euclidean')
        regressor2= LinearRegression()
        regressor3=LinearSVR()
        
        #Step 3
        trained_model1 = regressor1.fit(X_train, 
                                 y_train)
        trained_model2 = regressor2.fit(X_train, 
                                 y_train)
        trained_model3 = regressor3.fit(X_train, 
                                 y_train)  
        
        #Step 4
        incomplete_2 = deepcopy(incomplete)
        incomplete_2.loc[:, incomplete.columns != i] = incomplete_2.loc[:, 
                                incomplete.columns != i].apply(lambda row: row.fillna(row.mean()), axis=1)

        #Step 5
        y_pred1 = regressor1.predict(X_test)
        y_pred2 = regressor2.predict(X_test)
        y_pred3 = regressor3.predict(X_test)
        
        #Step 6
        kn_errors.append(mean_squared_error(y_test, y_pred1))
        linear_errors.append(mean_squared_error(y_test, y_pred2))
        svr_errors.append(mean_squared_error(y_test, y_pred3))
        
        
         
    MSE= []
    
    #Step 7
    for i in range(0, len(complete.loc[:,'2007':'2017'].columns)):
        l = []
        l.extend((kn_errors[i], linear_errors[i], svr_errors[i]))
        
        if min(l) == kn_errors[i]:
            MSE.append("KNN")
        elif min(l) == linear_errors[i]:
            MSE.append("Linear")
        elif min(l) == svr_errors[i]:
            MSE.append("SVR")

    #Step 8
    print("KNN =",MSE.count("KNN"),'\nLinear =',MSE.count("Linear") ,'\nSVR =',MSE.count("SVR"))
    return max(set(MSE), key = MSE.count)


##### 1.2.2 apply_regressor
The apply_regressor function uses the string return of regressor_test function to apply the chosen model. First, we check which model was chosen on the choice variable, following for almost the same steps of regressor_test instead for the last part:<br><br>
<span style="color:blue">
1 - Check which regressor we will use and create the regressor.
2 - Split the data in Train and Test (80% and 20% respectively)<br>
3 - Train each regressor with Train data.<br>
4 - Fill other columns (instead of the one in the loop) with the row average. We decided to do it because we cant have NaNs on the other columns.<br>
5 - Make the Predictions on the test set.<br>
6 and 7 - Fill the NaNs in incomplete and pass it to the main dataset.<br>
8 - Return the dataset filled.


In [3]:
def apply_regressor(choice,complete, incomplete,dataset,years):
    ''' The main goal of apply_regressor function is to apply the model chosen on regressor_test on the dataset.
    choice: String with name of the best model.
    Complete: Pandas Dataset with complete data.
    Incomplete: Pandas Dataset with NaNs.
    dataset: The main dataset with all the data (the one you want to fill the NaNs).
    years: List with the name of the columns'''
    
    #Step 1
    for i in years:
        if choice == 'KNN':
            regressor = KNeighborsRegressor(2, 
                                            weights ='distance', 
                                            metric = 'euclidean')
        elif choice == 'SVR':
            regressor = LinearSVR()
        elif choice == 'Linear':
            regressor = LinearRegression()
        #Step 2    
        X_train, X_test, y_train, y_test = train_test_split(complete.loc[:,complete.columns != i].values,
                                                            complete.loc[:,i].values, test_size = 0.2, random_state = 0)
        #Step 3
        trained_model = regressor.fit(X_train, 
                                 y_train)
        #Step 4
        incomplete_2 = deepcopy(incomplete)
        incomplete_2.loc[:, incomplete.columns != i] = incomplete_2.loc[:, 
                                incomplete.columns != i].apply(lambda row: row.fillna(row.mean()), axis=1)
        #Step 5
        prediction = trained_model.predict(incomplete_2.loc[:,incomplete_2.columns != i])
        temp_df = pd.DataFrame(prediction.reshape(-1,1), columns = [i])
        
        #Step 6
        #now we are filling incomplete 
        for index in range(len(temp_df)):
            if np.isnan(incomplete[i][index]):
                incomplete[i][index] = temp_df[i][index]


    #Step 7
    #and filling the nan's on the main dataframe
    dataset.loc[:,'2007':'2017'] = pd.concat([complete, incomplete])
    
    #Step 8
    return dataset


##### 1.2.3 download_clean_data
This function download the dataset and metadata, merge and clean it.<br>
***Approach:***

- Load Data.
- Select Country Name, Country Code and the 11 last years (2017 is used just to check the growth from 2017 to 2018);
- Join with metadata on Country Code;
- Set Country Name as Index;
- Removing the 'Regions' (Will just use countries)
- Drop rows with more than 3 NaNs


In [4]:
def download_clean_data(data_url, metadata_url):
    ''' Download the data from the given URL, merge with metadata, drop rows with more than 3 NaNs.
    data_url: Pandas main dataset URL
    metadata_url: Pandas metadata dataset URL '''
    
    #Read the dataset CSV
    data=pd.read_csv(data_url, header=2)
    
    #Select the columns with usefull data we need 2007 here to calculate the grow from 2007 to 2008
    data=data[['Country Name', 'Country Code','2007' , '2008', '2009', '2010', '2011',
                       '2012', '2013', '2014', '2015', '2016', '2017']]
        
        #Load metadata CSV
    metadata_country=pd.read_csv(metadata_url, header=0) 
    
        #Merge data CSV with Metadata CSV
    data_df=data.merge(metadata_country, on='Country Code', how='left') 
        
        #Set Country Name as Index
    new_index = data_df['Country Name']
    data_df.set_index(new_index,inplace=True )
    
        #Create a column named Is_Country for later removing the "areas" like asia
    data_df['Is_Country'] = data_df['Region'].notnull()
    
        #Drop unnecessary columns
    data_df.drop(['Country Name', 'Unnamed: 5', 'Region', 'IncomeGroup', 'SpecialNotes',
                  'TableName'], inplace=True, axis=1)
    
        #drop the 'areas'
    data_df = data_df[data_df.Is_Country != False]
    
        #drop the Is_Country column becouse we dont need it anymore
    data_df.drop('Is_Country', inplace=True, axis=1)
    
        #drop rows with 3 or more NANs values
    data_df.dropna(thresh=(len(data_df.loc[:,'2008':'2017'].columns) - 1), inplace=True, axis=0)
    

    return data_df

#### 1.2 Get URL's
We work with provided data set indicator name: international tourism, the number of arrivals, metadata country and another indicator name: international tourism, receipts (current US$) *changed to Income in the report*, from the same the world bank databank. 

We use GitHub for sharing files, brief note and set our next goals and importing libraries for data analysis and visualization. 

In [53]:
#Define the URI from the github files (Number of Arrivels and Income)
#If you have any problem with this code, just go to https://github.com/rennanvoa2/Programming_DS_Project
#Copy the raw link from each csv file and change the strings below
income_url = 'https://raw.githubusercontent.com/rennanvoa2/Programming_DS_Project/master/Income.csv?token=AGBCKJWJQHIXYVN4UUSPPKC5ZKGIY'
arrival_url = 'https://raw.githubusercontent.com/rennanvoa2/Programming_DS_Project/master/International%20Arrivals.csv?token=AGBCKJSNMRBUHCWG5W7FRG25ZKGNC'
metadata_url = 'https://raw.githubusercontent.com/rennanvoa2/Programming_DS_Project/master/Metadata_Country.csv?token=AGBCKJSUCSZUTPS4I5AECTS5ZKGOC'

#### 1.3 Load and Clean the Data
##### 1.3.1 Apply download_clean_data function.

In [54]:
arrivals_df = download_clean_data(arrival_url,metadata_url)
income_df=download_clean_data(income_url,metadata_url)

##### 1.3.2 Get the intersection
Dif_row is a merged data frame that only has common rows of information, merged by Country Code.<br>
We used this approach because we dropped some countries from both data frames (Arrivals and Income) when removed NaNs, and after that, the data frames have different countries. We decided to use just the countries that are in both data frames.<br>
Both data frames have the same 117 observations and 11 years (2007-2017) for next stage of Data Analysis.

In [55]:
#list of the names of the columns we are going to use
years = ['2007','2008','2009','2010','2011','2012','2013','2014', '2015','2016', '2017']

#dif_row is a merged dataframe that only has common rows of information, merged by Country Code
dif_row=pd.merge(arrivals_df,income_df,on='Country Code',left_index=True)
dif_row=dif_row.drop(['Country Code'], axis=1)

#We now have the same countries on both lists
arrivals_df=dif_row.iloc[:,0:int(len(dif_row.columns)/2)]
income_df=dif_row.iloc[:,int(len(dif_row.columns)/2):]

arrivals_df.head()
#renaming columns with the correct names
arrivals_df.columns = years
income_df.columns = years


#### 1.4 Split Arrival Data
Here we split the Arrivals data frame into data_arrivals_complete (rows without NaNs) and data_arrivals_incomplete (rows with at least one NaN)


In [56]:
 #preparing arrival data for regressors
data_arrivals_complete = pd.DataFrame()
data_arrivals_incomplete = arrivals_df[arrivals_df.isna().any(axis=1)]
data_arrivals_complete = arrivals_df[~arrivals_df.isna().any(axis=1)]

#### 1.5 Apply the best regressor on Arrival data frame.
We set the name of the best regressor on the variable choice (String). Then we apply this regressor with apply_regressor function and save the return data frame on arrivals_df.

In [57]:
#applying created functions
choice= regressor_test(data_arrivals_complete,data_arrivals_incomplete,years)
print('Best type of regression to be used for arrivals prediction->',choice)
arrivals_df=apply_regressor(choice,data_arrivals_complete,data_arrivals_incomplete,arrivals_df,years)


KNN = 0 
Linear = 8 
SVR = 3
Best type of regression to be used for arrivals prediction-> Linear


#### 1.6 Split Income Data
Here we split the Income data frame into data_income_complete (rows without NaNs) and data_income_incomplete (rows with at least one NaN)

In [58]:
#preparing income data for regressors
data_income_complete = pd.DataFrame()
data_income_incomplete = income_df[income_df.isna().any(axis=1)]
data_income_complete = income_df[~income_df.isna().any(axis=1)]


#### 1.7 Apply the best regressor on Income data frame..
After running the regressor_test function for the Income data frame, we realized that the regressor wasn't performing well. When we apply it, some predictions returned negative numbers. Then we decided to use the second best, which was SVR. 

In [59]:
#applying created functions
choice= regressor_test(data_income_complete,data_income_incomplete,years)
print('Best type of regression to be used for income prediction ->',choice)
income_df=apply_regressor('SVR', data_income_complete, data_income_incomplete, income_df, years)

KNN = 1 
Linear = 5 
SVR = 5
Best type of regression to be used for income prediction -> Linear


### 2. Data Preparation
#### 2.1 common_fm

The goal for this function is to create commom features in both datasets. We are using 4 commom features: 
- Avarage in 10 years(Avg_10_Years) - Mean of the row;
- %Avarage(%Avg) - divide each value of Avarage in 10 years for the sum of the column;
- Growth in 10 Years(Growth10ys)  - (Data from 2017 - data from 2008) / data from 2008;
- %Growth(% growth) - Divide each value of Growth in 10 years for the sum of the column.

In [60]:
def common_fm(data_df):
    ''' This function create features on the given dataset. 
    Features created: Avarage in 10 years(Avg_10_Years), Growth in 10 Years(Growth10ys), %Growth(% growth) and %Avarage(%Avg).
    data_df: Pandas Dataset'''
        #create feature Avarage in Last 10 Years
    data_df['Avg_10_Years'] = data_df.loc[:,'2008':'2017'].mean(axis=1)
    
        #Create the feature Growth in 10 years
    data_df['Growth10ys']=(data_df['2017']/data_df['2008']-1)
    
    
    annual_growth = pd.DataFrame(index=data_df.index.values)
    
        #Fill the growth of each year in annual_arrival_growth dataframe
    for i in arrivals_df.loc[:,'2008':'2017'].columns:
        annual_growth[i] = (data_df[i] - data_df[str(int(i)-1)]) / data_df[str(int(i)-1)]
       
        #New Growth metric, becouse the last one wasnt good.
    data_df["AVG_Growth"] =  annual_growth.mean(axis=1)
    
        #sort by the best avarage arrivals in the last 10 years
    data_df = data_df.sort_values('Avg_10_Years', ascending =False)

        #divide each value of Growth in 10 years for the sum of the column
    data_df['% growth'] = data_df['AVG_Growth'] / data_df['AVG_Growth'].sum()

        #divide each value of Avarage in 10 years for the sum of the column
    data_df['%Avg'] = data_df['Avg_10_Years'] / data_df['Avg_10_Years'].sum()


    return data_df

#### 2.2 Creating the new metrics
We started by giving the same weigh for growth metrics and total number metrics, but the result showed the countries with higher growth in the last 10 years dominated the top 10 countries despite the number of the total number. Then we decided to give more weight to the total number. It makes sense to balance both features, but the total number should count more than growth for a safe result.
We also created the feature AVG_expenditure_per_person, which was made dividing Average 10 years (income) for Average 10 years (arrivals), then we used this metric to create %Avg_Per_Person dividing each value for column total.
On our last step, we create Growth x Average x Avg Exp feature, which balances %Growth, %Avarage and %Avg Exp Feature based on the wight's chosen.

In [61]:
#Weigh's for metrics
arrivals_total_number_weight = 3
arrivals_growth_weight = 1
income_total_number_weight = 3
income_growth_weight = 1
avg_per_person_weight = 1


#for arrivals

arrivals_df = common_fm(arrivals_df)

#Calculate the avarage between Growth and Avarage Numbers of Arrivals
arrivals_df['Growth x Average'] = (arrivals_growth_weight * arrivals_df['% growth'] +
               (arrivals_total_number_weight * arrivals_df['%Avg'])) / (arrivals_total_number_weight + arrivals_growth_weight)

#create a dataframe sorted by Growth X Avarage
Arrivals_in_growth_vs_arrivals = arrivals_df.sort_values('Growth x Average', ascending=False)


#for income

income_df = common_fm(income_df)

#Create Avarage expenditure per person
income_df['AVG_expenditure_per_person'] = income_df['Avg_10_Years'] / arrivals_df['Avg_10_Years']


#divide each value of Avarage per person for the sum of the column
income_df['%Avg_Per_Person'] = income_df['AVG_expenditure_per_person'] / income_df['AVG_expenditure_per_person'].sum()

#Calculate the avarage between Growth and Avarage Numbers of Arrivals
income_df['Growth x Average x Avg Exp'] = (income_growth_weight * income_df['% growth'] +
               (income_total_number_weight* income_df['%Avg']) + 
               avg_per_person_weight * income_df['%Avg_Per_Person']) / (income_total_number_weight + 
                                                income_growth_weight + avg_per_person_weight)

#create a dataframe sorted by Growth X Avarage
income_in_growth_vs_income = income_df.sort_values('Growth x Average x Avg Exp', ascending=False)

#### 2.3 Drop Outliers
We didn't want to drop many outliers, but Belarus and Congo was biasing our data. No matter what weight we chose for total number, Belarus and Congo were always listed. When we check their growth on the dataframe the difference between them and the others was too big, raising a red flag. Checking the metadata we realized that data before 2015 were adjusted to reflect the new denomination effective from July 1, thats why we decided to drop it.


In [62]:
#Drop Belarus, its an outlier in Arrivals Dataset
Arrivals_in_growth_vs_arrivals = Arrivals_in_growth_vs_arrivals.drop(['Belarus'])

#Drop Congo, its an outlier in income Dataset
income_in_growth_vs_income = income_in_growth_vs_income.drop(['Congo, Dem. Rep.'])

#### 2.3 Pick the top 10 of each data frame
Using the Arrivals_in_growth_vs_arrivals and income_in_growth_vs_income features we selected the top 10 countries.

In [63]:
#get the best 10 results in Arrivals
arrival_top_10 = Arrivals_in_growth_vs_arrivals.iloc[0:10,:]

#get the best 10 results in income
income_top_10 = income_in_growth_vs_income.iloc[0:10,:]

### 3. Plots

Using Seaborn library we built the next bar charts showing the top countries with the biggest growth in arrivals, Income and a combination of both measures.
#### 3.1 Arrivals top 10 Bar Chart

This Bar Chart shows the 10 countries with the biggest average growth in Arrivals 

In [47]:
# Set some initial parameters of the graph
plt.rcParams['figure.figsize'] = (30, 10)
sb.set(style="whitegrid", font_scale=1.8)

# Plot the Arrivals values
sb.set_color_codes("pastel")
sb.barplot(y= arrival_top_10['Growth x Avarage'] , x = arrival_top_10.index.values, data = arrival_top_10, color='b')

# Name the x axis and set its fontsize
plt.xlabel('Top Countries', fontsize=25) 

# Name the y axis and set its fontsize
plt.ylabel('Percentage of Growth', fontsize=25) 
  
# Give a title and set its fontsize
plt.title("Growth in Arrivals", fontsize=25) 

KeyError: 'Growth x Avarage'

#### 3.2 Income top 10 Bar Chart

This Bar Chart shows the 10 countries with the biggest average growth in Income

In [48]:
# Set some initial parameters of the graph
plt.rcParams['figure.figsize'] = (30, 10)
sb.set(style="whitegrid", font_scale=1.8)

# Plot the Income values
sb.set_color_codes("muted")
sb.barplot(y= income_top_10['Growth x Avarage x Avg Exp'] , x = income_top_10.index.values, data = income_top_10, color='b')

# Name the x axis and set its fontsize
plt.xlabel('Top Countries', fontsize=25) 

# Name the y axis and set its fontsize
plt.ylabel('Percentage of Growth', fontsize=25) 
  
# Give a title and set its fontsize
plt.title("Growth in Income", fontsize=25) 

KeyError: 'Growth x Avarage x Avg Exp'

#### 3.2 Growth in Income and Arrivals Bar Chart

This graph is the combination of both Bar Charts above and shows the countries with the biggest growth in arrivals and income in the last years.

In [ ]:
#Create a list with unique countries in arrival_top_10 + income_top_10
label_array = list(set(list(arrival_top_10.index.values)+ list(income_top_10.index.values)))

#Create a dataframe with the sum of top arrivals + top income
Arrivals_plus_income = pd.DataFrame(index=label_array)
Arrivals_plus_income['Arrivals GxA'] = arrivals_df['Growth x Avarage']
Arrivals_plus_income['Income GxA'] = income_df['Growth x Avarage x Avg Exp']

#sort the DataFrame by income
Arrivals_plus_income = Arrivals_plus_income.sort_values('Income GxA', ascending =False)


# Set some initial parameters of the graph
plt.rcParams['figure.figsize'] = (30, 10)
sb.set(style="whitegrid", font_scale=1.8)

# Plot the Income values
sb.set_color_codes("muted")
sb.barplot(y=Arrivals_plus_income['Income GxA'], x = Arrivals_plus_income.index.values, data = Arrivals_plus_income, color='b', label='Income')

# Plot the Arrivals values
sb.set_color_codes("pastel")
sb.barplot(y= Arrivals_plus_income['Arrivals GxA'] , x = Arrivals_plus_income.index.values, data = Arrivals_plus_income, color='b', bottom=Arrivals_plus_income['Income GxA'], label='Arrival')

# Name the x axis and set its fontsize
plt.xlabel('Top Countries', fontsize=25) 

# Name the y axis and set its fontsize
plt.ylabel('Percentage of Growth', fontsize=25) 
  
# Give a title and set its fontsize
plt.title("Growth by Income and Arrivals", fontsize=25) 

# Add the legend
plt.legend()